In [1]:
import requests
import pandas as pd

In [2]:
csv = pd.read_csv('delitos_2020.csv')

In [3]:
csv.head()

,id_mapa,anio,mes,dia,fecha,franja,tipo,subtipo,uso_armas,barrio,comuna,latitud,longitud,victimas
0,RH-2020-0000480,2020,Enero,Miércoles,01/01/20,10,Hurto (sin violencia),NaN,NaN,Versalles,10.0,-34.626641,-58.527606,NaN
1,HD-2020-0001119,2020,Enero,Miércoles,01/01/20,22,Homicidio,Femicidio,NaN,Villa Devoto,11.0,-34.611268,-58.525386,1.0
2,RH-2020-0001140,2020,Enero,Miércoles,01/01/20,19,Robo (con violencia),NaN,NaN,Versalles,10.0,-34.625434,-58.525013,NaN
3,RH-2020-0001187,2020,Enero,Miércoles,01/01/20,17,Robo (con violencia),NaN,NaN,Versalles,10.0,-34.625434,-58.525013,NaN
4,RH-2020-0010817,2020,Enero,Miércoles,01/01/20,2,Hurto (sin violencia),NaN,NaN,Villa Devoto,11.0,-34.606789,-58.520726,NaN


In [4]:
# TIPOS
csv['tipo'].unique()

array(['Hurto (sin violencia)', 'Homicidio', 'Robo (con violencia)',
       'Lesiones'], dtype=object)

In [34]:
# TABLA TIEMPOS
tiempos_csv = csv[['anio', 'mes', 'dia', 'fecha', 'franja']]
tiempos_csv['nro_dia'] = tiempos_csv['fecha'].apply(lambda x: int(x.split('/')[0]))
tiempos_csv = tiempos_csv.drop_duplicates()
tiempos_csv['id'] = tiempos_csv.index + 1
tiempos_csv

/var/folders/zk/0yzb9y8s6z71smfld5fjvdjh0000gn/T/ipykernel_86634/1244753612.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tiempos_csv['nro_dia'] = tiempos_csv['fecha'].apply(lambda x: int(x.split('/')[0]))


,anio,mes,dia,fecha,franja,nro_dia,id
0,2020,Enero,Miércoles,01/01/20,10,1,1
1,2020,Enero,Miércoles,01/01/20,22,1,2
2,2020,Enero,Miércoles,01/01/20,19,1,3
3,2020,Enero,Miércoles,01/01/20,17,1,4
4,2020,Enero,Miércoles,01/01/20,2,1,5
...,...,...,...,...,...,...,...
66634,2020,Diciembre,Jueves,31/12/20,20,31,66635
66635,2020,Diciembre,Jueves,31/12/20,5,31,66636
66645,2020,Diciembre,Jueves,31/12/20,12,31,66646
66678,2020,Diciembre,Jueves,31/12/20,6,31,66679


In [31]:
# TABLA LUGARES
lugares_csv = csv[['barrio', 'comuna']]
# drop dups
lugares_csv = lugares_csv.drop_duplicates()
# add id column
lugares_csv['id'] = lugares_csv.index + 1
lugares_csv

,barrio,comuna,id
0,Versalles,10.0,1
1,Villa Devoto,11.0,2
5,Mataderos,9.0,6
6,Villa Luro,10.0,7
7,Monte Castro,10.0,8
15,Villa Urquiza,12.0,16
18,Vélez Sársfield,10.0,19
21,Saavedra,12.0,22
23,Agronomía,15.0,24
24,Villa Lugano,8.0,25


In [33]:
# TABLA TIPO DE ARMA
armas_csv = csv[['uso_armas']]
armas_csv = armas_csv.drop_duplicates(ignore_index=True)
armas_csv['id'] = armas_csv.index + 1
armas_csv

,uso_armas,id
0,NaN,1
1,Arma cortante,2
2,Arma de fuego,3


In [47]:
# TABLA DELITOS
delitos_csv = csv[['victimas', 'barrio', 'uso_armas', 'anio', 'mes', 'dia', 'franja']]

# replace barrio for lugar_id
delitos_csv = delitos_csv.merge(lugares_csv, left_on='barrio', right_on='barrio')
delitos_csv = delitos_csv.rename(columns={'id': 'lugar_id'})
delitos_csv = delitos_csv.drop(columns=['barrio', 'comuna'])

# replace uso_armas for arma_id
delitos_csv = delitos_csv.merge(armas_csv, left_on='uso_armas', right_on='uso_armas')
delitos_csv = delitos_csv.rename(columns={'id': 'arma_id'})
delitos_csv = delitos_csv.drop(columns=['uso_armas'])

# replace fecha, franja for tiempo_id
delitos_csv = delitos_csv.merge(tiempos_csv, left_on=['anio', 'mes', 'dia', 'franja'], right_on=['anio', 'mes', 'dia', 'franja'])
delitos_csv = delitos_csv.rename(columns={'id': 'tiempo_id'})
delitos_csv = delitos_csv.drop(columns=['anio', 'mes', 'dia', 'franja', 'nro_dia', 'fecha'])
delitos_csv

,victimas,lugar_id,arma_id,tiempo_id
0,NaN,1,1,1
1,NaN,1,1,2138
2,NaN,1,1,4396
3,NaN,1,1,6566
4,NaN,1,1,8861
...,...,...,...,...
290055,NaN,90,3,26901
290056,NaN,205,2,25612
290057,NaN,205,2,26901
290058,NaN,173,3,26857
